In [ ]:
import pickle
import pandas as pd
from tqdm import tqdm
import math

In [ ]:
def compute_info_keywords(listkeywords, dfkeywords, my_years, my_months):
    dico_importance_keywords = {'keyword': [], 'year': [], 'month': [], 'importance_for_the_month': [],
                                'importance_to_the_month': [],
                                'importance_for_the_year': [], 'importance_to_the_year': []}

    df_myinfos_keyword = dfkeywords.loc[dfkeywords['keyword'].isin(listkeywords)].copy()

    for keyword in listkeywords:
        mypreviousyearimportance = 0
        myincrementalvalue = 0

        infokeyword = df_myinfos_keyword.loc[df_myinfos_keyword['keyword'] == keyword].copy()
        for year in my_years:
            # I reduce my selection of citing paper to the year I am considering
            # this will make everything run much faster!
            infokeywords_year = infokeyword.loc[infokeyword['year'] == year].copy()
            for month in my_months:
                infokeywords_month = infokeywords_year.loc[infokeywords_year['month'] == month].copy()
                # we compute the number of citations for this specific month
                if len(infokeywords_month) == 0:
                    cos_sim = 0
                else:
                    list_cos_sim = infokeywords_month.cosine_similarity.tolist()
                    cos_sim = list_cos_sim[0]
                myincrementalvalue = myincrementalvalue + cos_sim
                dico_importance_keywords['importance_to_the_month'].append(myincrementalvalue)
                dico_importance_keywords['importance_for_the_month'].append(cos_sim)
                dico_importance_keywords['keyword'].append(keyword)
                dico_importance_keywords['year'].append(year)
                dico_importance_keywords['month'].append(month)
                if month == 'December':
                    # I add 12 times my incremental value
                    dico_importance_keywords['importance_to_the_year'] = \
                    dico_importance_keywords['importance_to_the_year'] + 12 * [myincrementalvalue]

                    importancefortheyear = myincrementalvalue - mypreviousyearimportance
                    dico_importance_keywords['importance_for_the_year'] = \
                    dico_importance_keywords['importance_for_the_year'] + 12 * [importancefortheyear]
                    mypreviousyearimportance = myincrementalvalue

    dfkeywords_importance = pd.DataFrame(dico_importance_keywords)
    return dfkeywords_importance

In [ ]:
infile_keywords = open('data_creation_variables/dfkeywords','rb')
dfkeywords = pickle.load(infile_keywords)
infile_keywords.close()

In [ ]:
# I create a list of all keywords I have in my dataset and their importance in general
mykeywords = list(set(dfkeywords.keyword.tolist()))

In [ ]:
# I create here a dataframe where I have the importance of each keyword in this
dico_importance_keywords={'keyword':[],'year':[],'month':[],'importance_for_the_month':[],
                          'importance_to_the_month':[],
                         'importance_for_the_year':[],'importance_to_the_year':[]}

In [ ]:
my_months = ['January','February','March','April','May','June',
             'July','August','September','October','November','December']

my_years = [2002,2003,2004,2005,2006,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022]

In [ ]:
numberkeywords = len(mykeywords)

fractionnumber = 100

list_listkeywords = []

In [ ]:
for i in tqdm(range(fractionnumber)):
    start = i*math.floor(numberkeywords/fractionnumber)
    end = (i+1)*math.floor(numberkeywords/fractionnumber)
    if i == fractionnumber-1:
        mylistkeywords= mykeywords[start:]
    else:
        mylistkeywords = mykeywords[start:end]
    list_listkeywords.append(mylistkeywords)

In [ ]:
listindex= [0,20,40,60,80,100]

In [ ]:
items = ['keyword','year','month','importance_for_the_month',
'importance_to_the_month', 'importance_for_the_year','importance_to_the_year']

In [ ]:
for j in range(5):
    dico_importance_keywords = {'keyword': [], 'year': [], 'month': [], 'importance_for_the_month': [],
                                'importance_to_the_month': [],
                                'importance_for_the_year': [], 'importance_to_the_year': []}

    list_listkeywords_compute=list_listkeywords[listindex[j]:listindex[j+1]]
    list_df = list(map(lambda x: compute_info_keywords(x,dfkeywords,my_years,my_months), tqdm(list_listkeywords_compute)))
    for element in list_df:
        for item in items:
            mynewlist = element[item].tolist()
            dico_importance_keywords[item] = dico_importance_keywords[item] + mynewlist
    newfile = pd.DataFrame(dico_importance_keywords)
    newfile.to_pickle('data_creation_variables/df_importance_keywords'+str(j))

In [ ]:
dico_importance_keywords = {'keyword': [], 'year': [], 'month': [], 'importance_for_the_month': [],
                            'importance_to_the_month': [],
                            'importance_for_the_year': [], 'importance_to_the_year': []}

In [ ]:
for j in range(5):
    with open('data_creation_variables/df_importance_keywords'+str(j), 'rb') as f:
        df_importance_keywords = pickle.load(f)
    for item in items:
        mynewlist = df_importance_keywords[item].tolist()
        dico_importance_keywords[item] = dico_importance_keywords[item] + mynewlist

In [ ]:
dfkeywords_importance = pd.DataFrame(dico_importance_keywords)
dfkeywords_importance.to_pickle('data_creation_variables/dfkeywords_importance')

In [ ]:
# create a panda data frame with : dico_average_cos_sim = {'keyword': [], 'average_sim': []

In [ ]:
mykeywords = list(set(dfkeywords.keyword.tolist()))

In [ ]:
dico_average_cos_sim = {'keyword': [], 'average_sim': []}

In [ ]:
for word in tqdm(mykeywords):
    info_word = dfkeywords.loc[dfkeywords['keyword'] == word].copy()
    info_cos_sim = info_word.cosine_similarity.tolist()
    average_sim = sum(info_cos_sim) / len(info_cos_sim)
    dico_average_cos_sim['keyword'].append(word)
    dico_average_cos_sim['average_sim'].append(average_sim)

In [ ]:
average_cos_sim_df = pd.DataFrame(dico_average_cos_sim)

In [ ]:
average_cos_sim_df.to_pickle('data_creation_variables/df_average_cosim')